# Data Cleaning

## Load libraries

In [2]:
# Import required packages
import pandas as pd
import re
import numpy as np

## Load data

In [4]:
# Load data and save to poems
poems = pd.read_parquet('datasets/archive/poems-with-targets-04-02-2022.parquet')


## Title and Poem columns

In [5]:
# Look at first line of poems dataframe
poems.head(1)

,title,poem,poet,tags,nationality,dates,sex,number_of_lines,number_of_stanzas
0,\r\r\n Objects Used to Prop Open a Window\r\r\n,"\r\r\nDog bone, stapler,\r\r\ncribbage board, garlic press\r\r\n because this window is loose—lacks\r\r\nsuction, lacks grip.\r\r\nBungee cord, bootstrap,\r\r\ndog leash, leather belt\r\r\n because this window had sash cords.\r\r\nThey frayed. They broke.\r\r\nFeather duster, thatch of straw, empty\r\r\nbottle of Elmer's glue\r\r\n because this window is loud—its hinges clack\r\r\nopen, clack shut.\r\r\nStuffed bear, baby blanket,\r\r\nsingle crib newel\r\r\n because this window is split. It's dividing\r\r\nin two.\r\r\nVelvet moss, sagebrush,\r\r\nwillow branch, robin's wing\r\r\n because this window, it's pane-less. It's only\r\r\na frame of air.\r\r\n",Michelle Menting,None,None,NaN,1.0,20,1


In [6]:
# Remove '\r\r\n' from title column
poems['title'] = poems.title.apply(lambda x: re.sub(r'\n','',re.sub(r'\r','',x)).strip()).copy()

In [7]:
# Remove '\r\r\n' from poem column
poems['poem'] = poems.poem.apply(lambda x: re.sub(r'\n',' ',re.sub(r'\r','',x))).copy()

In [8]:
# Look at first line of poems dataframe after removing '\r\r\n'
poems.head(1)

,title,poem,poet,tags,nationality,dates,sex,number_of_lines,number_of_stanzas
0,Objects Used to Prop Open a Window,"Dog bone, stapler, cribbage board, garlic press because this window is loose—lacks suction, lacks grip. Bungee cord, bootstrap, dog leash, leather belt because this window had sash cords. They frayed. They broke. Feather duster, thatch of straw, empty bottle of Elmer's glue because this window is loud—its hinges clack open, clack shut. Stuffed bear, baby blanket, single crib newel because this window is split. It's dividing in two. Velvet moss, sagebrush, willow branch, robin's wing because this window, it's pane-less. It's only a frame of air.",Michelle Menting,None,None,NaN,1.0,20,1


## Tags column

In [9]:
# Check number of unique values in tags column
len(poems.tags.unique())

8285

In [10]:
# Check unique tags
poems.tags.unique()

array([None,
       'Living,Time & Brevity,Relationships,Family & Ancestors,Nature,Landscapes & Pastorals,Seas, Rivers, & Streams,Social Commentaries,History & Politics',
       'Religion,The Spiritual,Mythology & Folklore,Fairy-tales & Legends',
       ...,
       'Activities,Eating & Drinking,Religion,Christianity,Other Religions,The Spiritual,Arts & Sciences,Reading & Books,Social Commentaries,History & Politics,Mythology & Folklore,Thanksgiving',
       'Living,Growing Old,Marriage & Companionship,Relationships,Home Life',
       'Nature,Animals,Seas, Rivers, & Streams,Summer,Winter,Religion,Arts & Sciences,Philosophy,Poetry & Poets'],
      dtype=object)

The above list shows me the tag column contains strings of all the tags assigned to a given poem. Each possible tag will need to be converted to a column header with 1,0,np.nan values populated for each poem as appropriate.

### One hot encode tags column

In [11]:
# Initialize list to track tags
all_tags = []

# Loop through each unique element in tags column
for tags in poems.tags.unique():
    # Use try-except to check for None value
    try:
        # Split tag string on commas to find individual tags
        for tag in tags.split(','):
            # Check if tag is 'Seas', 'Rivers' or '& Streams' to identify if 'Seas, Rivers, & Streams' tag is present
            if tag.strip() in ['Seas','Rivers','& Streams']:
                # Append 'Seas, Rivers & Streams tag to all tags
                all_tags.append('Seas, Rivers & Streams')
            else:
                # Append tag to all tags
                all_tags.append(tag.strip())
    # If None, move on to next element
    except:
        pass

In [12]:
# Convert all tags to set to remove duplicates and back to list
all_tags = list(set(all_tags))
all_tags

['Birth',
 'Romantic Love',
 'Activities',
 'Get Well & Recovery',
 'Music',
 'Indoor Activities',
 'The Spiritual',
 'Birth & Birthdays',
 'Living',
 'Thanksgiving',
 'Faith & Doubt',
 'Animals',
 'Gratitude & Apologies',
 'Health & Illness',
 'Youth',
 'Sciences',
 'The Mind',
 'Other Religions',
 'Death',
 'Theater & Dance',
 'War & Conflict',
 'Friends & Enemies',
 'Heroes & Patriotism',
 'Classic Love',
 'Growing Old',
 'Seas, Rivers & Streams',
 'Poetry & Poets',
 'Yom Kippur',
 'Winter',
 'Heavens',
 'Infancy',
 'Fairy-tales & Legends',
 'Farewells & Good Luck',
 'Travels & Journeys',
 'Halloween',
 'Race & Ethnicity',
 'Painting & Sculpture',
 'Labor Day',
 "Father's Day",
 'Disappointment & Failure',
 'Humor & Satire',
 'Family & Ancestors',
 'Summer',
 'Men & Women',
 'Funerals',
 'The Body',
 'Weather',
 'Coming of Age',
 'Anniversary',
 'Realistic & Complicated',
 'Spring',
 'Relationships',
 'September 11th',
 'God & the Divine',
 'Town & Country Life',
 'Nature',
 'Memori

In [13]:
# Define function to create dictionary mapping all possible tags to present tags per poem
# Input: tag string for poem
# Output: dictionary of all possible tags with present tag values set to 1

def check_tags(tags):
    # Use try-except to check for None value
    try:
        # Create dictionary with elements in all tags as keys, check if element is in tags and update value to 1 if it is else set value to 0
        all_tag_markup = {tag: 1 if tag in tags else 0 for tag in all_tags}

        # Return above dictionary
        return all_tag_markup
    except:
        # If None is provided, set value for every tag in dictionary to np.nan (Not a number)
        all_tag_markup = {tag: np.nan for tag in all_tags}

        # Return above dictionary
        return all_tag_markup

In [14]:
# Apply check tags to tags column and extract as a list of dictionaries
# Convert List of dictionaries to pandas dataframe for inspection
pd.DataFrame(list(poems['tags'].apply(check_tags).values))

,Birth,Romantic Love,Activities,Get Well & Recovery,Music,Indoor Activities,The Spiritual,Birth & Birthdays,Living,Thanksgiving,...,Marriage & Companionship,Love,Unrequited Love,Time & Brevity,Queer,St. Patrick's Day,Gender & Sexuality,Independence Day,Fall,Valentine's Day
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13754,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13755,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Concatenate tag dataframe shown above to original poems dataframe and drop original tags column
poems = pd.concat([poems.reset_index(drop=True),pd.DataFrame(list(poems['tags'].apply(check_tags).values))],axis=1).drop('tags',axis=1)

In [16]:
# Show first line of updated dataframe
poems.head(1)

,title,poem,poet,nationality,dates,sex,number_of_lines,number_of_stanzas,Birth,Romantic Love,...,Marriage & Companionship,Love,Unrequited Love,Time & Brevity,Queer,St. Patrick's Day,Gender & Sexuality,Independence Day,Fall,Valentine's Day
0,Objects Used to Prop Open a Window,"Dog bone, stapler, cribbage board, garlic press because this window is loose—lacks suction, lacks grip. Bungee cord, bootstrap, dog leash, leather belt because this window had sash cords. They frayed. They broke. Feather duster, thatch of straw, empty bottle of Elmer's glue because this window is loud—its hinges clack open, clack shut. Stuffed bear, baby blanket, single crib newel because this window is split. It's dividing in two. Velvet moss, sagebrush, willow branch, robin's wing because this window, it's pane-less. It's only a frame of air.",Michelle Menting,None,NaN,1.0,20,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Convert columns to snake case

In [18]:
# Define function to convert column headers to snake case (Number of Lines --> number_of_lines)
# Input: dataframe
# Output: snake_cased columns
def snake_case(df):
    # Create new list where for every column name in dataframe, replace commas and spaces with underscore (',' & ' ' --> '_') and replace ampersand with 'and' ('&' --> 'and')
    # Save new list as df.columns
    df.columns = [x.lower().replace(' ', '_').replace('&', 'and').replace(',','_') for x in df.columns]

    # Return df.columns
    return df.columns

In [19]:
# Convert poems columns to snake case
snake_case(poems)

Index(['title', 'poem', 'poet', 'nationality', 'dates', 'sex',
       'number_of_lines', 'number_of_stanzas', 'birth', 'romantic_love',
       ...
       'marriage_and_companionship', 'love', 'unrequited_love',
       'time_and_brevity', 'queer', 'st._patrick's_day',
       'gender_and_sexuality', 'independence_day', 'fall', 'valentine's_day'],
      dtype='object', length=135)

## Rename nationality column to country_of_origin

In [21]:
# Rename nationality column to country_of_origin
poems.rename(columns = {'nationality': 'country_of_origin'}, inplace=True)

## Check dtypes for target variables

- `sex`
- `country_of_origin`
- `dates`

In [22]:
# Create list defining target columns
targets = ['sex','country_of_origin','dates']

# Get information about target columns (data type, null counts)
poems[targets].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13759 entries, 0 to 13758
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sex                13759 non-null  object 
 1   country_of_origin  8002 non-null   object 
 2   dates              7577 non-null   float64
dtypes: float64(1), object(2)
memory usage: 322.6+ KB


### `sex`

`sex` has been cast as an `object`. I will check the unique values in this column:

In [25]:
# Check unique columns in sex column
poems.sex.unique()

array(['1.0', '0.0', 'female', 'male', 'other'], dtype=object)

We have different dermacations for sex in this column, these need to be standardized:

1.0 = female

0.0 = male

In [26]:
# Standardize values in sex column
poems.sex = poems.sex.apply(lambda x: 'female' if x in ['1.0','female'] else 'male' if x in ['0.0','male'] else None).copy()

In [27]:
# Check unique values in sex column
poems.sex.unique()

array(['female', 'male', None], dtype=object)

In [28]:
# Check info about target columns again
poems[targets].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13759 entries, 0 to 13758
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sex                13727 non-null  object 
 1   country_of_origin  8002 non-null   object 
 2   dates              7577 non-null   float64
dtypes: float64(1), object(2)
memory usage: 322.6+ KB


All target columns are the correct data type now. The final dataframe can be saved.

In [5]:
# poems = pd.read_parquet('datasets/poems-final.parquet')
poems.to_parquet('datasets/poems-final.parquet', index=False)

## Build Markdown Data Dictionary table

In [21]:
# For loop to build markdown table for data dictionary with column definitions
# Output to be pasted in .md file

# Iterate through every column in poems tracking the index
for index, column in enumerate(poems.columns):
    # Confirm if column index is before one-hot encoded ags columns
    if index < 8: 
        # Create markdown text for each column including column name, description to be manually updated, number of null rows, number of non-null rows
        print(f'|{column}|desc|{poems[column].dtypes}|{poems[column].isnull().sum()}|{len(poems[column]) - poems[column].isnull().sum()}|')
    else:
        # Create markdown text for each column including column name, description, number of null rows, number of non-null rows
        print(f'|{column}|Indicates if `{column}` tag has been applied to poem. <n>1: `{column}` tag has been applied<\\n><n>0: `{column}` tag has not been applied.<\\n>|{poems[column].dtypes}|{poems[column].isnull().sum()}|{len(poems[column]) - poems[column].isnull().sum()}|') 

|title|desc|object|0|13759|
|poem|desc|object|0|13759|
|poet|desc|object|0|13759|
|country_of_origin|desc|object|5757|8002|
|dates|desc|float64|6182|7577|
|sex|desc|float64|32|13727|
|number_of_lines|desc|int64|0|13759|
|number_of_stanzas|desc|int64|0|13759|
|sorrow_and_grieving|Indicates if `sorrow_and_grieving` tag has been applied to poem. <n>1: `sorrow_and_grieving` tag has been applied<\n><n>0: `sorrow_and_grieving` tag has not been applied.<\n>|float64|902|12857|
|war_and_conflict|Indicates if `war_and_conflict` tag has been applied to poem. <n>1: `war_and_conflict` tag has been applied<\n><n>0: `war_and_conflict` tag has not been applied.<\n>|float64|902|12857|
|easter|Indicates if `easter` tag has been applied to poem. <n>1: `easter` tag has been applied<\n><n>0: `easter` tag has not been applied.<\n>|float64|902|12857|
|town_and_country_life|Indicates if `town_and_country_life` tag has been applied to poem. <n>1: `town_and_country_life` tag has been applied<\n><n>0: `town_and_